In [1]:
import sys
sys.path.insert(0,'../')
sys.path

['../',
 'D:\\Thesis\\MachineFairness\\ContexualFairness\\2022-1Spring\\MissingValueDisparity\\notebooks',
 'c:\\python38_64\\python38.zip',
 'c:\\python38_64\\DLLs',
 'c:\\python38_64\\lib',
 'c:\\python38_64',
 '',
 'C:\\Users\\Rakin\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'c:\\python38_64\\lib\\site-packages',
 'c:\\python38_64\\lib\\site-packages\\pgmpy-0.1.15-py3.8.egg',
 'c:\\python38_64\\lib\\site-packages\\win32',
 'c:\\python38_64\\lib\\site-packages\\win32\\lib',
 'c:\\python38_64\\lib\\site-packages\\Pythonwin',
 'c:\\python38_64\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Rakin\\.ipython']

In [2]:
import numpy as np
from utils import *
from datasets.fair_dataset import FairDataset

In [3]:
protected = ["sex"]
privileged_classes = [['Male']]
class_shift = 10
kwargs = {
    'protected_attribute_names': ['sex'],
    'privileged_group': 'Male',
    'favorable_class': 1,
    'classes': [0, 1],
    'sensitive_groups': ['Female', 'Male'],
    'group_shift': 0,
    'alpha': 0.25, 'beta': 1,
    'dist': {'mus': {1: np.array([10, 15]), 
                     0: np.array([10 - class_shift, 15 - class_shift])}, 'sigmas': [3, 3]}
}

In [4]:
estimator = get_estimator('nb', False)
keep_prot = False

In [5]:
def get_performance_summary(train_fd, test_fd, estimator, prediction_func):
    pmod, pmod_results = get_groupwise_performance(train_fd, test_fd,
                                               estimator,
                                               privileged=True,
                                               pos_rate=False)
    umod, umod_results = get_groupwise_performance(train_fd, test_fd,
                                               estimator,
                                               privileged=False,
                                               pos_rate=False)
    mod, mod_results = get_groupwise_performance(train_fd, test_fd,
                                             estimator,
                                             privileged=None,
                                             pos_rate=False)

    p_perf = get_model_performances(pmod, test_fd,
                                prediction_func, keep_prot=keep_prot)
    u_perf = get_model_performances(umod, test_fd,
                                prediction_func, keep_prot=keep_prot)
    m_perf = get_model_performances(mod, test_fd,
                                prediction_func, keep_prot=keep_prot)
    print_table_row(is_header=False, var_value='mean', p_perf=p_perf,
                        u_perf=u_perf, m_perf=m_perf, variable='method')
    return pmod, umod, mod, p_perf, u_perf, m_perf

In [6]:
train_fd, test_fd = get_datasets(
    type='ds_ccd', n_samples=10000, n_features=2,
    train_random_state=47, test_random_state=41,
    **kwargs, method='simple_imputer.mean')

{'n_redline': 1, 'random_seed': 47, 'n_samples': 10000, 'n_features': 2, 'protected_attribute_names': ['sex'], 'privileged_group': 'Male', 'favorable_class': 1, 'classes': [0, 1], 'sensitive_groups': ['Female', 'Male'], 'group_shift': 0, 'alpha': 0.25, 'beta': 1, 'dist': {'mus': {1: array([10, 15]), 0: array([0, 5])}, 'sigmas': [3, 3]}, 'method': 'simple_imputer.mean'}
{'n_redline': 1, 'random_seed': 41, 'imputer': SimpleImputer(), 'n_samples': 10000, 'n_features': 2, 'protected_attribute_names': ['sex'], 'privileged_group': 'Male', 'favorable_class': 1, 'classes': [0, 1], 'sensitive_groups': ['Female', 'Male'], 'group_shift': 0, 'alpha': 0.25, 'beta': 1, 'dist': {'mus': {1: array([10, 15]), 0: array([0, 5])}, 'sigmas': [3, 3]}, 'method': 'simple_imputer.mean'}


Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


               0          1  sex  label
0       4.187863        NaN  0.0    0.0
1       5.751583   2.224400  0.0    0.0
2       6.701431   1.879459  0.0    0.0
3       4.538972   7.369555  0.0    0.0
4       1.321352   2.155979  0.0    0.0
...          ...        ...  ...    ...
24995   8.061156   8.729148  1.0    1.0
24996   3.200909  10.595855  1.0    1.0
24997  12.299590  10.174539  1.0    1.0
24998   9.108419  10.880650  1.0    1.0
24999  15.231006   5.475872  1.0    1.0

[25000 rows x 4 columns]
['0', '1']
               0          1
0       4.187863        NaN
1       5.751583   2.224400
2       6.701431   1.879459
3       4.538972   7.369555
4       1.321352   2.155979
...          ...        ...
24995   8.061156   8.729148
24996   3.200909  10.595855
24997  12.299590  10.174539
24998   9.108419  10.880650
24999  15.231006   5.475872

[25000 rows x 2 columns]


Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [7]:
complete_df = train_fd.complete_df.astype(float)

In [8]:
train_fd.group_configs

[([5.0, 5], [3, 3], 0, 0),
 ([0, 10.0], [3, 3], 1, 0),
 ([5.0, 15], [3, 3], 0, 1),
 ([10, 10.0], [3, 3], 1, 1)]

In [9]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(complete_df[complete_df.columns[:-1]], complete_df[complete_df.columns[-1]])
model.theta_

array([[ 2.52454351,  7.49482017,  0.5       ],
       [ 7.51887714, 12.51922959,  0.5       ]])

In [10]:
model.sigma_

array([[15.36399082, 15.43459251,  0.25000002],
       [15.01558608, 14.62920217,  0.25000002]])

In [11]:
baseline_fd = FairDataset(df=complete_df, protected_attribute_names=['sex'])

In [12]:
for s in [0, 1]:
    for l in [0, 1]:
        print('s', s, 'label', l)
        print(complete_df[(complete_df['sex'] == s) & (complete_df['label'] == l)].describe())

s 0 label 0
                  0             1      sex    label
count  10000.000000  10000.000000  10000.0  10000.0
mean       5.028427      4.957250      0.0      0.0
std        3.027555      2.996059      0.0      0.0
min       -7.679162     -6.693075      0.0      0.0
25%        2.968444      2.887698      0.0      0.0
50%        5.034149      4.930117      0.0      0.0
75%        7.082838      6.955998      0.0      0.0
max       17.733290     15.507203      0.0      0.0
s 0 label 1
                 0            1     sex   label
count  2500.000000  2500.000000  2500.0  2500.0
mean      5.040839    15.009477     0.0     1.0
std       2.980293     2.913850     0.0     0.0
min      -6.494888     5.767241     0.0     1.0
25%       3.019072    13.018414     0.0     1.0
50%       5.064649    14.997273     0.0     1.0
75%       7.034455    16.995055     0.0     1.0
max      14.436139    24.943924     0.0     1.0
s 1 label 0
                  0             1      sex    label
count  10000

In [13]:
incomplete_df = train_fd.get_incomplete_df()
for s in [0, 1]:
    for l in [0, 1]:
        print('s', s, 'label', l)
        print(incomplete_df[(incomplete_df['sex'] == s) & (incomplete_df['label'] == l)].describe())

s 0 label 0
                  0            1      sex    label
count  10000.000000  6014.000000  10000.0  10000.0
mean       5.028427     4.937781      0.0      0.0
std        3.027555     2.990918      0.0      0.0
min       -7.679162    -6.693075      0.0      0.0
25%        2.968444     2.847601      0.0      0.0
50%        5.034149     4.901025      0.0      0.0
75%        7.082838     6.902566      0.0      0.0
max       17.733290    14.880912      0.0      0.0
s 0 label 1
                 0            1     sex   label
count  2500.000000  1486.000000  2500.0  2500.0
mean      5.040839    15.011042     0.0     1.0
std       2.980293     2.924892     0.0     0.0
min      -6.494888     6.262150     0.0     1.0
25%       3.019072    12.992444     0.0     1.0
50%       5.064649    14.970256     0.0     1.0
75%       7.034455    17.075720     0.0     1.0
max      14.436139    24.692377     0.0     1.0
s 1 label 0
                  0            1      sex    label
count  10000.000000  8

In [14]:
train_fd.imputer.statistics_

array([3.52341024, 8.79376228])

In [15]:
test_fd.imputer.statistics_

array([3.52341024, 8.79376228])

In [16]:
train_fd.get_incomplete_df()['1'].count()

18750

In [17]:
print_table_row(is_header=True, variable='Method')
output = get_performance_summary(baseline_fd, baseline_fd, estimator, get_predictions)
output = get_performance_summary(train_fd, train_fd, estimator, get_predictions)

Method	 & 	AC_p	 & 	AC_u	 & 	SR_p	 & 	SR_u	 & 	FPR_p	 & 	FPR_u
mean	 & 	96.3	 & 	96.6	 & 	12.5	 & 	12.6	 & 	01.3	 & 	01.5
mean	 & 	96.3	 & 	89.7	 & 	12.5	 & 	08.8	 & 	00.9	 & 	03.1


In [18]:
pmod, umod, mod, _, _, _ = output

In [19]:
pmod.theta_

array([[0.02065958, 9.90927692],
       [9.99691537, 9.91210032]])

In [20]:
pmod.sigma_

array([[9.02393924, 8.26541501],
       [8.87123236, 7.64553229]])

In [21]:
umod.theta_

array([[ 5.02842743,  6.47477541],
       [ 5.04083892, 12.48931311]])

In [22]:
umod.sigma_

array([[ 9.16517299,  8.94325381],
       [ 8.87859289, 14.40084233]])

In [23]:
mod.theta_

array([[ 2.52454351,  8.19202616],
       [ 7.51887714, 11.20070672]])

In [24]:
mod.sigma_

array([[15.36399082, 11.55328457],
       [15.01558608, 12.68369375]])